In [1]:
!nvidia-smi
!which python | grep DYY

Sun Sep 22 21:08:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000004:04:00.0 Off |                    0 |
| N/A   44C    P0              60W / 184W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Model

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SinousEmbedding(nn.Module):
    def __init__(self, dim) -> None:
        super().__init__()
        assert dim%2==0,NotImplementedError()
        self.angles = (10000.**(-2/dim))**torch.arange(1,dim//2+1,1,dtype=torch.float).cuda()
        self.angles.requires_grad_(False)

    def forward(self,x):
        angles = torch.einsum('m,i->im',self.angles,x.float())
        return torch.cat((torch.sin(angles),torch.cos(angles)),dim=1)
    
class Attn(nn.Module):
    def __init__(self,dim,head):
        super().__init__()
        assert dim%head==0,NotImplementedError()
        self.head = head
        self.head_dim = dim // head
        self.Q = nn.Linear(dim,dim,bias=False)
        self.K = nn.Linear(dim,dim,bias=False)
        self.V = nn.Linear(dim,dim,bias=False)
        self.out_proj = nn.Linear(dim,dim)
        self.apply_init()

    def apply_init(self):
        self.Q.weight.data.normal_(0,0.02)
        self.K.weight.data.normal_(0,0.02)
        self.V.weight.data.normal_(0,0.02)
        self.out_proj.weight.data.zero_()
        self.out_proj.bias.data.zero_()
    
    def forward(self,query,context):
        q = self.Q(query).reshape(*query.shape[:2],self.head,self.head_dim)
        k = self.K(context).reshape(*context.shape[:2],self.head,self.head_dim)
        v = self.V(context).reshape(*context.shape[:2],self.head,self.head_dim)
        score = torch.einsum('bihd,bjhd->bijh',q,k) / (self.head_dim**0.5)
        score = F.softmax(score,dim=2)
        out = torch.einsum('bijh,bjhd->bihd',score,v).reshape_as(query)
        return self.out_proj(out)
    
class GELU(nn.Module):
    def forward(self,x):
        return 0.5 * x * (1 + torch.tanh(0.7978845608 * (x + 0.044715 * x**3)))
    
class SiLU(nn.Module):
    def forward(self,x):
        return x * torch.sigmoid(x)
    
class AdaptiveLayerNorm(nn.Module):

    def __init__(self, dim):
        super().__init__()
        self.norm = nn.LayerNorm(dim,elementwise_affine=False)
        self.beta_gamma = nn.Sequential(
            SiLU(),
            nn.Linear(dim,2*dim)
        )
        self.beta_gamma[-1].weight.data.zero_()
        self.beta_gamma[-1].bias.data.zero_()

    def forward(self, x, conditioned):
        x = self.norm(x)
        beta,gamma = self.beta_gamma(conditioned).unsqueeze(1).chunk(2,dim=-1)
        return x * (1+gamma) + beta

class Layer(nn.Module):

    def __init__(self,dim,head):
        super().__init__()
        self.attn = Attn(dim,head)
        self.mlp = nn.Sequential(
            nn.Linear(dim,dim),
            GELU(),
            nn.Linear(dim,dim)
        )
        self.condition_mlp = nn.Sequential(
            nn.Linear(dim,dim),
            SiLU(),
            nn.Linear(dim,2*dim)
        )
        self.norm1 = AdaptiveLayerNorm(dim)
        self.norm2 = AdaptiveLayerNorm(dim)
        self.apply_init()

    def apply_init(self):
        self.condition_mlp[2].weight.data.zero_()
        self.condition_mlp[2].bias.data.zero_()

    def forward(self,x,condition):
        alpha1,alpha2 = self.condition_mlp(condition).unsqueeze(1).chunk(2,dim=-1)

        # first half
        xc = x.clone()
        x = self.norm1(x,condition)
        x = self.attn(x,x)
        x = x * alpha1
        x = x + xc

        # second half
        xc = x.clone()
        x = self.norm2(x,condition)
        x = self.mlp(x)
        x = x * alpha2
        x = x + xc

        return x

class DiT(nn.Module):

    def __init__(self,
            patch_size=4,
            hidden_dim=256,
            num_layers=6,
            image_size = 28*28,
            num_heads=8
        ):
        super().__init__()
        self.num_patches = image_size // (patch_size * patch_size)
        self.embedding = nn.Linear(patch_size * patch_size, hidden_dim, bias=False)
        self.pos_embedding = SinousEmbedding(hidden_dim)
        self.t_embedding = nn.Sequential(
            SinousEmbedding(64),
            nn.Linear(64,hidden_dim),
            SiLU(),
            nn.Linear(hidden_dim,hidden_dim)
        )
        self.patch_size = patch_size
        self.num_layers = num_layers
        self.layers = nn.ModuleList([Layer(hidden_dim,num_heads) for _ in range(num_layers)])
        self.out_norm = AdaptiveLayerNorm(hidden_dim)
        # self.out_norm = nn.LayerNorm(hidden_dim)
        self.out_proj = nn.Linear(hidden_dim, patch_size * patch_size, bias=False)
        self.apply_init()

    def apply_init(self):
        self.embedding.weight.data.normal_(0,0.02)
        self.out_proj.weight.data.zero_()

    def first(self,x,t):
        t_embed = self.t_embedding(t)
        # patchify the image x
        x = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size) # [B, 1, H/4, W/4, 4, 4]
        x_embed = self.embedding(x.reshape(x.shape[0],-1,self.patch_size*self.patch_size))
        # print('x range:',x_embed.min(),x_embed.max(),x_embed.std())
        x_embed += self.pos_embedding(torch.arange(x_embed.shape[1],device=device))
        # print('position embedding range:',x_embed.min(),x_embed.max(),x_embed.std())
        data = torch.cat((x_embed,t_embed.unsqueeze(1)),dim=1)
        # print('position embedding range:',pos_embed.min(),pos_embed.max(),pos_embed.std())
        # print('x range:',data.min(),data.max(),data.std())
        return data, t_embed
    
    def forward(self,x,t):
        x = x.reshape(x.shape[0],1,28,28)
        inputs, conditioned = self.first(x,t)
        for i,ly in enumerate(self.layers):
            inputs = ly(inputs,conditioned)
            # print('layer',i,'input.range:',inputs.min(),inputs.max())
        # remove t token
        inputs = inputs[:,:-1,:]
        inputs = self.out_norm(inputs,conditioned)
        # print('output range:',inputs.min(),inputs.max())
        inputs = self.out_proj(inputs)
        # print('output range:',inputs.min(),inputs.max())
        # patchify the image x
        length = 28 // self.patch_size
        inputs = inputs.reshape(inputs.shape[0],length,length,self.patch_size,self.patch_size).permute(0,3,4,1,2).reshape(inputs.shape[0],self.patch_size*self.patch_size,length*length)
        x = F.fold(inputs, (28,28), self.patch_size, stride=self.patch_size)
        return x.reshape(x.shape[0],-1)

# Train

In [3]:
import sys
import os

# parent_dir = os.path.abspath('/root/DeepLearning')
parent_dir = os.path.abspath('/home/zhh24/DeepLearning')
# parent_dir = os.path.abspath('..')

sys.path.append(parent_dir)
print('appended',parent_dir)

import utils

from tqdm import tqdm
import torch
import torch.nn.functional as F
import torchvision.utils

import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.autograd.set_detect_anomaly(True)
mnist = utils.MNIST(batch_size=256)
train_loader = mnist.train_dataloader
valid_loader = mnist.valid_dataloader
T=1000
# beta1=1e-4 # variance of lowest temperature
# betaT=6e-2 # variance of highest temperature
eps = 8e-3
steps=torch.linspace(0,T,steps=T+1,dtype=torch.float)
f_t=torch.cos(((steps/T+eps)/(1.0+eps))*3.14159*0.5)**2
betas=torch.clamp(1.0-f_t[1:]/f_t[:T],0.0,0.999)

# step = torch.log(torch.tensor(betaT/beta1))/(T-1)
# betas = beta1 * torch.exp(step*torch.arange(T,dtype=torch.float).to(device))
# step = (betaT-beta1)/(T-1)
# betas = torch.arange(T,dtype=torch.float,device=device) * step + beta1


alphas = 1-betas
alpha_bars = alphas.clone()
for i in range(1,T):
    alpha_bars[i] *= alpha_bars[i-1]

print(alpha_bars)
print('range of bars',alpha_bars.min(),alpha_bars.max())
# print(alphas)

sqrt = torch.sqrt
sigmas = sqrt(betas * (1-alpha_bars / alphas)/(1-alpha_bars))
sigmas[0] = 1
print('range of sigmas,',sigmas.min(),sigmas.max())
alphas = alphas.to(device)
alpha_bars = alpha_bars.to(device)
betas = betas.to(device)
sigmas = sigmas.to(device)
weights = torch.ones(T,dtype=torch.float,device=device)

@torch.no_grad()
def sample(model:DiT,save_dir):
    x = torch.randn([100,784]).to(device)
    for t in range(T-1,-1,-1):
        sigmaz = torch.randn_like(x)*sigmas[t]
        if t==0:
            sigmaz = 0
        noise_pred = model(x,t*torch.ones(x.shape[0],dtype=torch.long,device=device))
        x0_pred = (x - noise_pred * sqrt(1 - alpha_bars[t])) / sqrt(alpha_bars[t]).clamp(-1,1)
        mean_pred = (sqrt(alphas[t]) * (1-alpha_bars[t]/alphas[t]) * x + sqrt(alpha_bars[t]/alphas[t]) * (1-alphas[t]) * x0_pred) / (1-alpha_bars[t])
        x = mean_pred + sigmaz
        # x = torch.clamp(x,0,1)
    grid = torchvision.utils.make_grid(post_process(x).reshape(-1,1,28,28).cpu(), nrow=10)
    torchvision.utils.save_image(grid, save_dir)

@torch.no_grad()
def visualize(model,save_dir):
    interval = (T-1) // 20
    x = torch.randn([10,784]).to(device)
    x_history = []
    for t in range(T-1,-1,-1):
        sigmaz = torch.randn_like(x)*((betas[t])**0.5).to(device)
        if t==0:
            sigmaz = 0
        x = (x-(1-alphas[t])/(sqrt(1-alpha_bars[t]))*model(x,t*torch.ones(x.shape[0],dtype=torch.long,device=device)))/(sqrt(alphas[t]))+sigmaz
        # x = torch.clamp(x,0,1)
        x_history.append(x)
    # print('cat.shape',torch.cat(x_history,dim=0).shape)
    grid = torchvision.utils.make_grid(post_process(torch.stack(x_history,dim=0)[::interval,...]).reshape(-1,1,28,28).cpu(), nrow=10)
    torchvision.utils.save_image(grid, save_dir)
    print('Saved visualize to',os.path.abspath(save_dir))

@torch.no_grad()
def visualize_denoise(model,save_dir):
    # get 10 images from the dataset
    x,_ = next(iter(valid_loader))
    x = x[:20,...].reshape(20,784).to(device)
    x = pre_process(x)
    t = torch.tensor([i * T // 20 for i in range(20)],dtype=torch.long,device=device)
    noise = torch.randn_like(x).reshape(-1,784)
    v1 = (sqrt(alpha_bars[t]).reshape(-1,1)*x).reshape(-1,784)
    v2 = sqrt(1-alpha_bars[t]).reshape(-1,1)*noise
    x_corr = v1+v2
    est = model(x_corr,t)
    x_rec = (x_corr - sqrt(1-alpha_bars[t]).reshape(-1,1)*est)/(sqrt(alpha_bars[t])).reshape(-1,1)
    grid_orig = torchvision.utils.make_grid(post_process(x).reshape(-1,1,28,28).cpu(), nrow=10)
    grid_corr = torchvision.utils.make_grid(post_process(x_corr).reshape(-1,1,28,28).cpu(), nrow=10)
    grid_rec = torchvision.utils.make_grid(post_process(x_rec).reshape(-1,1,28,28).cpu(), nrow=10)
    # add noise level infomation to the image
    noise_level = (1-alpha_bars[t]).reshape(-1).tolist()
    ori_mse = noise.pow(2).mean(dim=1).reshape(-1).tolist()
    mse = ((est-noise)**2).mean(dim=1).reshape(-1).tolist()
    print(noise_level)
    print(ori_mse)
    print(mse)
    grid = torch.cat([grid_orig,grid_corr,grid_rec],dim=1)
    torchvision.utils.save_image(grid, save_dir)
    print('Saved denoise to',os.path.abspath(save_dir))

def plot_loss(losses,save_dir):
    losses_vals, t_vals = zip(*losses)
    losses_vals = torch.cat(losses_vals,dim=0)
    t_vals = torch.cat(t_vals,dim=0)

    results = []
    for t in range(T):
        this_t = abs(t_vals.float()-float(t))<0.5
        results.append(torch.sum(torch.where(this_t,losses_vals,torch.tensor(0.,device=device))).item() / (torch.sum(this_t.float())+1e-3).item())
    plt.plot(results)
    plt.ylim(0,max(results)* 1.2)
    plt.savefig(save_dir)
    plt.close()
    # weights = (torch.tensor(results,device=device)) # weights
    weights = torch.ones(T,dtype=torch.float,device=device)
    # weights[:10]=0
    # weights[10:80] /= 100
    return weights

def pre_process(x):
    return x*2-1

def post_process(x):
    return (x+1)/2

def train(epochs,model:DiT,optimizer,eval_interval=5):
    global weights
    for epoch in range(epochs):
        # print('weights normalized:',weights/weights.sum())
        all_ts = torch.distributions.Categorical(weights).sample((50000,))
        cnt = 0
        model.train()
        with tqdm(train_loader) as bar:
            losses = []
            for x,_ in bar:
                cnt += x.shape[0]
                x = pre_process(x.to(device))
                epss = torch.randn_like(x).reshape(-1,784).to(device)
                # ts = torch.randint(0,T,(x.shape[0],),device=device,dtype=torch.long)
                ts = all_ts[cnt-x.shape[0]:cnt]
                alpha_tbars = alpha_bars[ts]
                value = (sqrt(alpha_tbars).reshape(-1,1,1,1)*x).reshape(-1,784)+sqrt(1-alpha_tbars).reshape(-1,1)*epss
                out = model(value,ts) # [batch,784]
                # loss = ((epss-out).pow(2).mean(dim=-1) * (betas[ts])/(2*alphas[ts]*(1-alpha_tbars))).sum(dim=0)
                loss = ((epss-out).pow(2).mean(dim=-1)).mean(dim=0)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses.append(loss.item())
                bar.set_description('epoch {}, loss {:.4f}'.format(epoch,sum(losses)/len(losses)))

        model.eval()
        with torch.no_grad():
            with tqdm(valid_loader) as bar:
                mses = []
                losses = []
                losses_for_t = []
                for x,_ in bar:
                    x = pre_process(x.to(device))
                    epss = torch.randn_like(x).reshape(-1,784).to(device)
                    ts = torch.randint(0,T,(x.shape[0],),device=device,dtype=torch.long)
                    # print(ts)
                    alpha_tbars = alpha_bars[ts]
                    value = (sqrt(alpha_tbars).reshape(-1,1,1,1)*x).reshape(-1,784)+sqrt(1-alpha_tbars).reshape(-1,1)*epss
                    out = model(value,ts)
                    mse = F.mse_loss(epss,out)
                    mses.append(mse.item())
                    loss = ((epss-out).pow(2).mean(dim=-1))
                    # loss = (epss-out).pow(2).mean(dim=-1)
                    losses_for_t.append((loss.clone().detach(),ts))
                    loss = (loss).mean(dim=0)
                    losses.append(loss.item())
                    bar.set_description('epoch {}, MSE {:.4f}, [Valid] {:.4f}'.format(epoch,sum(mses)/len(mses),sum(losses)/len(losses)))
                    
        if epoch % eval_interval == 0:
            visualize(model,save_dir=os.path.join('./samples',f'diffuse_epoch_{epoch}.png'))
            sample(model,save_dir=os.path.join('./samples',f'sample_epoch_{epoch}.png'))
            # visualize_denoise(model,save_dir=os.path.join('./samples',f'denoise_epoch_{epoch}.png'))
            weights = plot_loss(losses_for_t,save_dir=os.path.join('./samples',f'loss_epoch_{epoch}.png'))
            torch.save(model,os.path.join('./samples',f'epoch_{epoch}.pt'))

if __name__ == '__main__':
    model = DiT().to(device)
    print('Number parameters of the model:', sum(p.numel() for p in model.parameters()))
    print('Model strcuture:',model)
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
    os.makedirs('./samples',exist_ok=True)
    # sample(model,save_dir=os.path.join('./samples',f'init.png'))
    # visualize(model,save_dir=os.path.join('./samples',f'init_visualize.png'))
    train(200,model,optimizer,eval_interval=5)

appended /home/zhh24/DeepLearning
tensor([9.9996e-01, 9.9991e-01, 9.9986e-01, 9.9981e-01, 9.9975e-01, 9.9968e-01,
        9.9961e-01, 9.9953e-01, 9.9945e-01, 9.9937e-01, 9.9928e-01, 9.9918e-01,
        9.9908e-01, 9.9898e-01, 9.9887e-01, 9.9876e-01, 9.9864e-01, 9.9851e-01,
        9.9839e-01, 9.9825e-01, 9.9811e-01, 9.9797e-01, 9.9782e-01, 9.9767e-01,
        9.9751e-01, 9.9735e-01, 9.9718e-01, 9.9701e-01, 9.9683e-01, 9.9665e-01,
        9.9647e-01, 9.9627e-01, 9.9608e-01, 9.9588e-01, 9.9567e-01, 9.9546e-01,
        9.9525e-01, 9.9502e-01, 9.9480e-01, 9.9457e-01, 9.9434e-01, 9.9410e-01,
        9.9385e-01, 9.9360e-01, 9.9335e-01, 9.9309e-01, 9.9283e-01, 9.9256e-01,
        9.9229e-01, 9.9201e-01, 9.9172e-01, 9.9144e-01, 9.9115e-01, 9.9085e-01,
        9.9055e-01, 9.9024e-01, 9.8993e-01, 9.8961e-01, 9.8929e-01, 9.8897e-01,
        9.8864e-01, 9.8830e-01, 9.8796e-01, 9.8762e-01, 9.8727e-01, 9.8691e-01,
        9.8656e-01, 9.8619e-01, 9.8582e-01, 9.8545e-01, 9.8507e-01, 9.8469e-01,
      

  0%|                                                                                                                                    | 0/188 [00:00<?, ?it/s]

Number parameters of the model: 5349376
Model strcuture: DiT(
  (embedding): Linear(in_features=16, out_features=256, bias=False)
  (pos_embedding): SinousEmbedding()
  (t_embedding): Sequential(
    (0): SinousEmbedding()
    (1): Linear(in_features=64, out_features=256, bias=True)
    (2): SiLU()
    (3): Linear(in_features=256, out_features=256, bias=True)
  )
  (layers): ModuleList(
    (0): Layer(
      (attn): Attn(
        (Q): Linear(in_features=256, out_features=256, bias=False)
        (K): Linear(in_features=256, out_features=256, bias=False)
        (V): Linear(in_features=256, out_features=256, bias=False)
        (out_proj): Linear(in_features=256, out_features=256, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): GELU()
        (2): Linear(in_features=256, out_features=256, bias=True)
      )
      (condition_mlp): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
      

epoch 0, MSE 0.1647, [Valid] 0.1647: 100%|███████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:03<00:00, 11.79it/s]


Saved visualize to /home/zhh24/samples/diffuse_epoch_0.png


...

Saved visualize to /home/zhh24/samples/diffuse_epoch_5.png


epoch 10, MSE 0.0995, [Valid] 0.0995: 100%|██████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:03<00:00, 11.77it/s]


Saved visualize to /home/zhh24/samples/diffuse_epoch_10.png


epoch 14, loss 0.0929:  48%|████████████████████████████████████████████████▍                                                   | 91/188 [01:01<01:06,  1.47it/s]

: 